In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.metrics import f1_score

In [3]:
### Constants
seed = 69
cv_size = .3

train = pd.read_csv("data/Train.csv", encoding="ISO-8859-1")
test = pd.read_csv("data/Test.csv", encoding="ISO-8859-1")

X = train.loc[:, train.columns != 'revenue']
y = train["revenue"]

# encode string input values as integers
features = []
encode_cols = ["month", "channelGrouping", "browser", "operatingSystem", "deviceCategory", "continent", 
                "subContinent", "country", "region", "metro", "city", "topLevelDomain", "campaign", "source", 
                "medium", "keyword", "referralPath", "adContent", "adwordsClickInfo.page", "adwordsClickInfo.slot",
                "adwordsClickInfo.adNetworkType", "bounces", "newVisits"
              ]

other_cols = ["visitNumber", "timeSinceLastVisit", "isMobile", "isTrueDirect", "hits", "pageviews"]

# encode string input values as integers
encoded_x = None
for i in encode_cols:
    col = X[i].apply(str)
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(col.fillna('0'))
    feature = feature.reshape(X.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False)
    feature = onehot_encoder.fit_transform(feature)
    if encoded_x is None:
        encoded_x = feature
    else:
        encoded_x = np.concatenate((encoded_x, feature), axis=1)
# print("X shape: ", encoded_x.shape)

X = np.concatenate((X[other_cols].fillna(0), encoded_x), axis=1)
y = Binarizer().transform(y.values.reshape(-1, 1))
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=cv_size, random_state=seed)

In [4]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [5]:
print(X.shape)
print(y.shape)

(83578, 2504)
(83578, 1)


In [6]:
epochs = 100
batch_size = 69

In [10]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(200, input_dim=2504, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model. We use the the logarithmic loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [11]:
model = create_model()

In [12]:
model.fit(X_train, y_train, epochs=epochs)#, batch_size=batch_size)

Epoch 1/100
58504/58504 [==============================] - 5s 82us/step - loss: 1.3419 - acc: 0.8912
Epoch 2/100
58504/58504 [==============================] - 3s 45us/step - loss: 0.8149 - acc: 0.9273
Epoch 3/100
58504/58504 [==============================] - 3s 43us/step - loss: 0.8079 - acc: 0.9285
Epoch 4/100
58504/58504 [==============================] - 3s 43us/step - loss: 0.8013 - acc: 0.9287
Epoch 5/100
58504/58504 [==============================] - 3s 43us/step - loss: 0.8146 - acc: 0.9284
Epoch 6/100
58504/58504 [==============================] - 3s 44us/step - loss: 0.7981 - acc: 0.9293
Epoch 7/100
58504/58504 [==============================] - 3s 45us/step - loss: 3.2629 - acc: 0.7752
Epoch 8/100
58504/58504 [==============================] - 3s 44us/step - loss: 0.8096 - acc: 0.9298
Epoch 9/100
58504/58504 [==============================] - 3s 45us/step - loss: 0.8018 - acc: 0.9298
Epoch 10/100
58504/58504 [==============================] - 3s 43us/step - loss: 0.8019 - a

In [18]:
pred = model.predict(X_train)

In [22]:
bin_pred = Binarizer(threshold=.5).transform(pred)

In [23]:
f1 = f1_score(y_train, bin_pred)
f1

0.521891418563923